In [1]:
# Introduce Weather colormaps to Plotly

In [2]:
import numpy as np
import pandas as pd
import plotly.express.colors as pxc
import xarray as xr
from collections import OrderedDict
from datetime import datetime

In [3]:
def ColorRampsWetterstation(fn):

    v = pd.read_csv(
        fn,
        sep=";",
        decimal=",",
        na_values=["unter", "Unter", "Und Tiefer", "Und tiefer"]
        + ["und höher", "Und höher"],
    )
    
    v

    # fix color values
    v = v.rename(columns={"Von": "From", "Bis": "To", "HEX Color": "HEX"})
    v["HEX"] = v["HEX"].apply(lambda x: "#{}".format(x))
    v["RGB"] = v["HEX"].apply(lambda x: pxc.hex_to_rgb(x))
    v["RGB_label"] = v["RGB"].apply(lambda x: pxc.label_rgb(x))
    v["RGB"] = v["RGB"].apply(lambda x: '{:03d}-{:03d}-{:03d}'.format(*x))

    # fix column names and values
    v["From"] = v["From"].fillna(-np.inf)
    v["To"] = v["To"].fillna(np.inf)

    v = v.set_index(["From", "To"]).sort_index().reset_index()
    return v


def ColorRampsWetterstation_to_xarray(table):
    tmp = table.reset_index()
    tmp = (
        pd.melt(
            tmp.reset_index(),
            value_vars=["To", "From"],
            value_name="variable_bounds",
            var_name="bounds",
            id_vars=[x for x in tmp.columns if x not in ["To", "From"]],
        )
        .set_index(["index", "bounds"])
        .to_xarray()
    )
    return tmp


def table_colorscale(table):
    scale = table["variable_bounds"].replace([np.inf, -np.inf], np.nan)
    vmin, vmax = (scale.min(axis=0).min(), scale.max(axis=0).max())
    vptp = vmax - vmin

    table["From_colorbreaks"] = scale["From"].fillna(vmin - vptp / 20)
    table["To_colorbreaks"] = scale["To"].fillna(vmax + vptp / 20)

    table["From_colorscale"] = np.round(
        (table["From_colorbreaks"] - (vmin - vptp / 20)) / (vptp + vptp / 20 * 2),
        decimals=3,
    )
    table["To_colorscale"] = np.round(
        (table["To_colorbreaks"] - (vmin - vptp / 20)) / (vptp + vptp / 20 * 2),
        decimals=3,
    )

    return table


def table_to_colorscale(table):
    cs = []
    for i, r in table.iterrows():
        for n in ["From_colorscale", "To_colorscale"]:
            cs.append([table.loc[i, n], pxc.label_rgb(r["RGB"])])
    return cs

In [4]:
colorramps = [
    {
        "name": "air pressure",
        "units": "mbar",
        "table": ColorRampsWetterstation(
            "../src/Color-Ramps-Wetterstation/Farbrampen-Luftdruck.csv"
        ),
    },
    {
        "name": "relative humidity",
        "units": "percent",
        "table": ColorRampsWetterstation(
            "../src/Color-Ramps-Wetterstation/Farbrampen-Relative Luftfeuchte (%).csv"
        ),
    },
    {
        "name": "global radiation",
        "units": "W m-2",
        "table": ColorRampsWetterstation(
            "../src/Color-Ramps-Wetterstation/Farbrampen-Globalstrahlung (W m2).csv"
        ),
    },
    {
        "name": "wind speed",
        "units": "m s-1",
        "table": ColorRampsWetterstation(
            "../src/Color-Ramps-Wetterstation/Farbrampen-Wind (m s).csv"
        ),
    },
    {
        "name": "air temperature",
        "units": "degrees_Celcius",
        "table": ColorRampsWetterstation(
            "../src/Color-Ramps-Wetterstation/Farbrampen-Lufttemperatur (°C).csv"
        ),
    },
    {
        "name": "precipitation",
        "units": "mm hr-1",
        "table": ColorRampsWetterstation(
            "../src/Color-Ramps-Wetterstation/Farbrampen-Niederschlag (mm hr).csv"
        ),
    },
]

In [5]:
# convert to xarray

tmp = [ColorRampsWetterstation_to_xarray(x["table"]) for x in colorramps]
ds = xr.concat(tmp, pd.Index([x["name"] for x in colorramps], name="name"))
# ds = ds.assign_coords(units=("name", [x["units"] for x in colorramps]))

In [6]:
# add color scale info (Plotly)

dx = []
for idx, group in ds.groupby("name"):
    tmp = group["variable_bounds"].where(np.isfinite(group["variable_bounds"]))
    vmin, vmax = (tmp.min(axis=0).min(), tmp.max(axis=0).max())
    vptp = vmax - vmin

    f = 20
    tmp.loc[:, :, "From"] = tmp.sel(bounds="From").fillna(vmin - vptp / f)
    tmp.loc[:, :, "To"] = tmp.sel(bounds="To").fillna(vmax + vptp / f)

    group = group.assign(variable_limits=(["name", "bounds"], [[vmin, vmax]]))
    group["colorscale_bounds"] = np.round(
        (tmp - (vmin - vptp / f)) / (vptp + vptp / f * 2),      
        decimals=3,
    )
    group = group.assign(
        colorscale_limits=(["name", "bounds"], [[vmin - vptp / f, vmax + vptp / f]])
    )

    dx.append(group)

dx = xr.combine_by_coords(dx)

In [7]:
globattr = OrderedDict(
    name="METEO color ramps",
    author="Andreas Christen",
    institution="University of Freiburg",
    license="Copyright (c) Andreas Christen, 2022.",
    history=(" ").join(
        [
            "[{}]".format(datetime.utcnow().strftime("%Y%m%dT%H%M%S+0000")),
            "Transcription of the ColorRampsWetterstation spreadsheet data.",
        ]
    ),
)

for k, v in globattr.items():
    dx.attrs[k] = v

In [8]:
dx['name']

<xarray.DataArray 'name' (name: 6)>
array(['air pressure', 'air temperature', 'global radiation', 'precipitation',
       'relative humidity', 'wind speed'], dtype=object)
Coordinates:
  * name     (name) object 'air pressure' 'air temperature' ... 'wind speed'

In [9]:
dx.sel(name='air temperature')

<xarray.Dataset>
Dimensions:            (index: 16, bounds: 2)
Coordinates:
  * index              (index) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * bounds             (bounds) object 'From' 'To'
    name               <U15 'air temperature'
Data variables:
    HEX                (index, bounds) object '#233D6B' '#233D6B' ... '#E31A15'
    RGB                (index, bounds) object '035-061-107' ... '227-026-021'
    RGB_label          (index, bounds) object 'rgb(35, 61, 107)' ... 'rgb(227...
    variable_bounds    (index, bounds) float64 -inf -6.0 -6.0 ... 31.0 31.0 inf
    variable_limits    (bounds) float64 -6.0 31.0
    colorscale_bounds  (index, bounds) float64 0.0 0.045 0.045 ... 0.955 1.0
    colorscale_limits  (bounds) float64 -7.85 32.85
Attributes:
    name:         METEO color ramps
    author:       Andreas Christen
    institution:  University of Freiburg
    license:      Copyright (c) Andreas Christen, 2022.
    history:      [20240716T131106+0000] Transcription of the ColorRampsWette...

In [10]:
dx.to_netcdf('../data/ColorRampsWetterstation.nc')